In [1]:
!conda --version

/bin/bash: line 1: conda: command not found


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install -q condacolab
import condacolab
condacolab.install()

⏬ Downloading https://github.com/conda-forge/miniforge/releases/download/23.11.0-0/Mambaforge-23.11.0-0-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:07
🔁 Restarting kernel...


In [4]:
!conda env update -n base -f /content/drive/MyDrive/CITS5553/environment.yml

Channels:
 - conda-forge
 - pytorch
Platform: linux-64
Solving environment: - \ | / - \ | / - \ done


==> WARNING: A newer version of conda exists. <==
    current version: 23.11.0
    latest version: 24.7.1

Please update conda by running

    $ conda update -n base -c conda-forge conda



pytorch-2.4.1        | 1.34 GB   | :   0% 0/1 [00:00<?, ?it/s]
torchtriton-3.0.0    | 233.4 MB  | :   0% 0/1 [00:00<?, ?it/s]

libcublas-12.4.2.65  | 228.4 MB  | :   0% 0/1 [00:00<?, ?it/s]


mkl-2022.1.0         | 199.6 MB  | :   0% 0/1 [00:00<?, ?it/s]



libcufft-11.2.0.44   | 166.9 MB  | :   0% 0/1 [00:00<?, ?it/s]




libcusparse-12.3.0.1 | 112.4 MB  | :   0% 0/1 [00:00<?, ?it/s]





libnpp-12.2.5.2      | 94.3 MB   | :   0% 0/1 [00:00<?, ?it/s]






libcusolver-11.6.0.9 | 78.1 MB   | :   0% 0/1 [00:00<?, ?it/s]







qt-main-5.15.8       | 58.4 MB   | :   0% 0/1 [00:00<?, ?it/s]








pillow-10.3.0        | 39.8 MB   | :   0% 0/1 [00:00<?, ?it/s]









libcurand-10

In [5]:
!pip install git+https://github.com/synsense/rockpool@develop

  Cloning https://github.com/synsense/rockpool (to revision develop) to /tmp/pip-req-build-_ax4_lfx
  Running command git clone --filter=blob:none --quiet https://github.com/synsense/rockpool /tmp/pip-req-build-_ax4_lfx
  Resolved https://github.com/synsense/rockpool to commit b97efd35f61babc190372fd0b844bd671f0c3f30
  Preparing metadata (setup.py) ... done
  Created wheel for rockpool: filename=rockpool-2.8.6.dev0-py3-none-any.whl size=604308 sha256=a22f101b5c7ca17d04625f4b46a97c353759bb2653f1e54ab82f0a3c8a1afb65
  Stored in directory: /tmp/pip-ephem-wheel-cache-m3g1_m4u/wheels/6e/b2/66/55f471c6fd9bafeeb0d6b2e3b193042877ed47cdd41da7a42c
Successfully built rockpool


In [4]:
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from rockpool.nn.networks import SynNet
from scipy import sparse
from torch.nn import MSELoss
from torch.optim import Adam
from torch.utils.data import DataLoader, Dataset
from tqdm.autonotebook import tqdm

/usr/local/lib/python3.10/site-packages/rockpool/nn/networks/__init__.py:15: UserWarning: This module needs to be ported to teh v2 API.
  warnings.warn(f"{err}")
/usr/local/lib/python3.10/site-packages/rockpool/nn/networks/__init__.py:20: UserWarning: This module needs to be ported to the v2 API.
  warnings.warn(f"{err}")
<ipython-input-4-a4230e0a0147>:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [5]:
# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [6]:
def raster(spikes: np.ndarray):
    spikes = np.asarray(spikes)
    dt = 1 / 110e3
    num_clk_per_period = 0.01 / dt
    spike_sum = np.cumsum(spikes, axis=0)[:: int(num_clk_per_period), :]
    spike_sum[1:, :] -= spike_sum[:-1, :]

    spike_sum[spike_sum > 15] = 15

    return spike_sum

In [7]:
def preload_samples(data, base_path, target_length):
    preloaded_samples = []

    for i, row in tqdm(data.iterrows(), total=len(data)):
        path = row['path']
        data_path = base_path + '/' + path
        sparse_spikes = sparse.load_npz(data_path)
        dense_spikes = sparse_spikes.toarray()
        raster_spikes = torch.Tensor(raster(dense_spikes))

        sample_length = raster_spikes.size(0)
        if sample_length < target_length:
            padding = target_length - sample_length
            raster_spikes = F.pad(raster_spikes, (0, 0, 0, padding))
        elif sample_length > target_length:
            start_idx = (sample_length - target_length) // 2
            end_idx = start_idx + target_length
            raster_spikes = raster_spikes[start_idx:end_idx]

        preloaded_samples.append(raster_spikes)

    data['tensor'] = preloaded_samples

    return data[['class','tensor']]


In [8]:
class SpikeDataset(Dataset):
    def __init__(self, annotations_file, split="train", target_length=200, alpha=1):
        self._label_map = {"car": 0, "cv": 1, "background": 2}
        self.spike_dir = os.path.dirname(annotations_file)
        self.split = split
        self.target_length = target_length
        self.alpha = alpha

        data = pd.read_csv(annotations_file)
        if split == "train":
            train_spiked = data[data["split"] == "train"]
            self.loaded_data = preload_samples(train_spiked, self.spike_dir, target_length)
        elif split == "val":
            val_spiked = data[data["split"] == "val"]
            self.loaded_data = preload_samples(val_spiked, self.spike_dir, target_length)
        elif split == "test":
            test_spiked = data[data["split"] == "test"]
            self.loaded_data = preload_samples(test_spiked, self.spike_dir, target_length)

    def __len__(self):
        return len(self.loaded_data)

    def __getitem__(self, idx):
        label = self.loaded_data.iloc[idx]['class']
        label_idx = self._label_map[label]
        label_tensor = torch.full((self.target_length, 3), -self.alpha) # change to -self.alpha if using vmem, 0 if not
        label_tensor[:, label_idx] = self.alpha

        raster_spikes = self.loaded_data.iloc[idx]['tensor']

        return raster_spikes, label_tensor.float()

In [9]:
train_dataset = SpikeDataset("/content/drive/MyDrive/CITS5553/data/spiked/loc1_500.csv", split="train")
val_dataset = SpikeDataset("/content/drive/MyDrive/CITS5553/data/spiked/loc1_500.csv", split="val")
test_dataset = SpikeDataset("/content/drive/MyDrive/CITS5553/data/spiked/loc1_500.csv", split="test")

  0%|          | 0/1049 [00:00<?, ?it/s]

<ipython-input-7-9642c6612d1d>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['tensor'] = preloaded_samples


  0%|          | 0/226 [00:00<?, ?it/s]

<ipython-input-7-9642c6612d1d>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['tensor'] = preloaded_samples


  0%|          | 0/225 [00:00<?, ?it/s]

<ipython-input-7-9642c6612d1d>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['tensor'] = preloaded_samples


In [10]:
train_dataset.alpha = 3
val_dataset.alpha = 3
test_dataset.alpha = 3

In [11]:
train_dl = DataLoader(train_dataset, batch_size=256, shuffle=True)
test_dl = DataLoader(test_dataset, batch_size=256, shuffle=True)
val_dl = DataLoader(val_dataset, batch_size=256, shuffle=True)

In [12]:
net = SynNet(
    output="vmem",

    n_channels=16,  # Number of input channels
    n_classes=3,  # Number of output classes (i.e. output channels)
    size_hidden_layers=[24, 24, 24],  # Number of neurons in each hidden layer
    time_constants_per_layer=[2, 4, 8],  # Number of time constants in each hidden layer
).to(device)

optimizer = Adam(net.parameters().astorch(), lr=1e-3)
loss_fun = MSELoss()

In [13]:
def accuracy(outputs, labels, alpha):
    mean_output = torch.mean(output, dim=1)
    pred = torch.argmin(torch.abs(mean_output - 1), dim=-1)

    mean_labels = torch.mean(labels, dim=1)
    target = torch.argmin(torch.abs(mean_labels - 1), dim=-1)

    correct_preds = (pred == target).float()

    return correct_preds.mean().item()

In [14]:
with tqdm(range(200), unit="epoch", desc="Epochs") as epoch_bar:
    for epoch in epoch_bar:
        net.train()

        total_loss = 0.0
        total_accuracy = 0.0
        total_batches = 0

        with tqdm(train_dl, desc=f"Epoch {epoch+1}", unit="batch", leave=False) as batch_bar:
            for events, labels in batch_bar:
                optimizer.zero_grad()

                events = events.to(device)
                labels = labels.to(device)

                output, _, _ = net(events)
                output = output.to(device)

                loss = loss_fun(output, labels)
                total_loss += loss.item()

                accuracy_ = accuracy(output, labels, 1)
                total_accuracy += accuracy_

                loss.backward()
                optimizer.step()

                total_batches += 1

                batch_bar.set_postfix({
                    "Loss": total_loss / total_batches,
                    "Accuracy": total_accuracy / total_batches
                })

        avg_loss = total_loss / total_batches
        avg_accuracy = total_accuracy / total_batches

        epoch_bar.set_postfix({
            "avg_loss": avg_loss,
            "avg_accuracy": avg_accuracy
        })


Epochs:   0%|          | 0/200 [00:00<?, ?epoch/s]

Epoch 1:   0%|          | 0/5 [00:00<?, ?batch/s]

Epoch 2:   0%|          | 0/5 [00:00<?, ?batch/s]

Epoch 3:   0%|          | 0/5 [00:00<?, ?batch/s]

Epoch 4:   0%|          | 0/5 [00:00<?, ?batch/s]

...

Epoch 196:   0%|          | 0/5 [00:00<?, ?batch/s]

Epoch 197:   0%|          | 0/5 [00:00<?, ?batch/s]

Epoch 198:   0%|          | 0/5 [00:00<?, ?batch/s]

Epoch 199:   0%|          | 0/5 [00:00<?, ?batch/s]

Epoch 200:   0%|          | 0/5 [00:00<?, ?batch/s]

In [15]:
import os

# Define the directory path
dir_path = "/content/drive/MyDrive/CITS5553/models"

# Create the directory if it doesn't exist
os.makedirs(dir_path, exist_ok=True)


In [16]:
torch.save(net.state_dict(), "/content/drive/MyDrive/CITS5553/models/initial.pt")

In [17]:
torch.save(train_dataset, "/content/drive/MyDrive/CITS5553/models/train.pt")
torch.save(val_dataset, "/content/drive/MyDrive/CITS5553/models/val.pt")
torch.save(test_dataset, "/content/drive/MyDrive/CITS5553/models/test.pt")

In [18]:
net.eval()

with tqdm(range(1), unit="epoch", desc="Epochs") as epoch_bar:
    for epoch in epoch_bar:

        total_loss = 0.0
        total_accuracy = 0.0
        total_batches = 0

        with tqdm(test_dl, desc=f"Epoch {epoch+1}", unit="batch", leave=False) as batch_bar:
            for events, labels in batch_bar:
                optimizer.zero_grad()

                events = events.to(device)
                labels = labels.to(device)

                output, _, _ = net(events)
                output = output.to(device)

                loss = loss_fun(output, labels)
                total_loss += loss.item()

                accuracy_ = accuracy(output, labels, 1)
                total_accuracy += accuracy_

                loss.backward()
                optimizer.step()

                total_batches += 1

                batch_bar.set_postfix({
                    "Loss": total_loss / total_batches,
                    "Accuracy": total_accuracy / total_batches
                })

        avg_loss = total_loss / total_batches
        avg_accuracy = total_accuracy / total_batches

        epoch_bar.set_postfix({
            "avg_loss": avg_loss,
            "avg_accuracy": avg_accuracy
        })


Epochs:   0%|          | 0/1 [00:00<?, ?epoch/s]

Epoch 1:   0%|          | 0/1 [00:00<?, ?batch/s]

In [19]:
%cd /content/drive/MyDrive/CITS5553

/content/drive/MyDrive/CITS5553


In [20]:
!git config --global user.name "KhabeerAhmed"
!git config --global user.email "khabeer1999@gmail.com"


In [ ]:
#git init
#git add .
#git commit -m "Initial commit with files"
#git branch -M main
#git remote add origin https://<your_access_token>@github.com/KhabeerAhmed/acoustic-traffic-monitoring.git
#git push -u origin main

In [21]:
%cd /content/drive/MyDrive/CITS5553

/content/drive/MyDrive/CITS5553


In [23]:
#Initialising DVC
!pip install dvc
!pip install dvc[gdrive]



  Using cached attrs-24.2.0-py3-none-any.whl.metadata (11 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.5/101.5 kB 5.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached rich-13.8.1-py3-none-any.whl.metadata (18 kB)
  Using cached tabulate-0.9.0-py3-none-any.whl.metadata (34 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 5.4 MB/s eta 0:00:00
  Using cached click-8.1.7-py3-none-any.whl.metadata (3.0 kB)
  Using cached typer-0.12.5-py3-none-any.whl.metadata (15 kB)
  Using cached pydantic-2.9.2-py3-none-any.whl.metadata (149 kB)
  Using cached entrypoints-0.4-py3-none-any.whl.metadata (2.6 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 12.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached markdown_it_py-3.0.0-py3-none-any.whl.metadata (6.9 kB)
  Using cached pygments-2.18.0-py3-none-any.whl.metadata (2.5 kB)


  Using cached PyDrive2-1.20.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached oauth2client-4.1.3-py2.py3-none-any.whl.metadata (1.2 kB)
  Using cached pyOpenSSL-24.2.1-py3-none-any.whl.metadata (13 kB)
  Using cached httplib2-0.22.0-py3-none-any.whl.metadata (2.6 kB)
  Using cached google_auth_httplib2-0.2.0-py2.py3-none-any.whl.metadata (2.2 kB)
  Using cached uritemplate-4.1.1-py2.py3-none-any.whl.metadata (2.9 kB)
  Using cached pyasn1-0.6.1-py3-none-any.whl.metadata (8.4 kB)
  Using cached pyasn1_modules-0.4.1-py3-none-any.whl.metadata (3.5 kB)
  Using cached rsa-4.9-py3-none-any.whl.metadata (4.2 kB)
  Using cached googleapis_common_protos-1.65.0-py2.py3-none-any.whl.metadata (1.5 kB)
  Using cached proto_plus-1.24.0-py3-none-any.whl.metadata (2.2 kB)
  Using cached cachetools-5.5.0-py3-none-any.whl.metadata (5.3 kB)
Using cached PyDrive2-1.20.0-py3-none-any.whl (47 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 108.4 MB/s eta 0:00:00
Using cached oauth2client-4.1